In [ ]:
import os
from dotenv import load_dotenv
import numpy as np
import tensorflow as tf
import pandas as pd
import datetime

load_dotenv(override=True)

DATA_PATH = os.getenv('DATA_PATH')
print(DATA_PATH)

## Reading fasta, obo and tsv files

In [ ]:
from Bio import SeqIO

sequences = [rec.seq for rec in SeqIO.parse(os.path.join(DATA_PATH, "Train/train_sequences.fasta"),"fasta")]
ids = [rec.id for rec in SeqIO.parse(os.path.join(DATA_PATH, "Train/train_sequences.fasta"),"fasta")]

In [ ]:
print("There are {} sequences in the dataset.".format(len(sequences)))

In [ ]:
import networkx
import obonet

# Read the taxrank ontology
url = os.path.join(DATA_PATH, "Train/go-basic.obo")
graph = obonet.read_obo(url)

# Number of nodes
print(len(graph))

# Number of edges
print(graph.number_of_edges())

# Check if the ontology is a DAG
print(networkx.is_directed_acyclic_graph(graph))


In [ ]:
df = pd.read_csv(os.path.join(DATA_PATH, "Train/train_terms.tsv"), sep='\t')
uniqueTerms = df["term"].unique()
termsArr = list(df["term"].to_numpy())

uniqueTermsDict={}
for i,el in enumerate(uniqueTerms):
    uniqueTermsDict[el] = i
    
termToken = [uniqueTermsDict[el] for el in termsArr]
df["termToken"] = termToken
df.head(10)

In [ ]:
df.shape

Test for the first entry:

In [ ]:
df.loc[df['EntryID'] == "A0A009IHW8"]

## GO analysis

In [ ]:
item_counts = df["term"].value_counts()
print(item_counts[0:30])

In [ ]:
id_to_name = {id_: data.get('name') for id_, data in graph.nodes(data=True)}
name_to_id = {data['name']: id_ for id_, data in graph.nodes(data=True) if 'name' in data}
print(id_to_name['GO:0005575'] )
print(id_to_name['GO:0008150'] )
print(id_to_name['GO:0110165'] )

In [ ]:
print(id_to_name['GO:0042324'] )
print(networkx.ancestors(graph, 'GO:0042324'))
print(networkx.descendants(graph, 'GO:0042324'))

paths = networkx.all_simple_paths(
    graph,
    source='GO:0042324',
    target=name_to_id['molecular_function']
)

for path in paths:
    print('•', ' ⟶ '.join(id_to_name[node] for node in path))

In [ ]:
allGOs= df.loc[df['EntryID'] == "A0A009IHW8"]["term"].to_numpy()
print([[id_to_name[el],el] for el in allGOs])

### Find GOs without ancestors

In [ ]:
sortedGOs = list(networkx.topological_sort(graph))
rootGOs = []
for g in sortedGOs:
    if len(networkx.ancestors(graph,g)) ==0:
        rootGOs.append(g)
    else:
        break
        
print(rootGOs)
print(len(rootGOs))

In [ ]:
print(networkx.ancestors(graph, sortedGOs[1000]))
print(id_to_name[sortedGOs[1000]])

### How many of them are used in our dataset?

In [ ]:
dataRootGOs = np.intersect1d(uniqueTerms,rootGOs)
print(len(dataRootGOs))

## Label encoding

The task is a multilabel classification: The output has several possible targets (Gene Ontologies) but each can only be 1 (existing) or 0 (non existing)

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

dftest=df.loc[df['EntryID'] == "A0A009IHW8"]
indices = dftest["termToken"].to_numpy()

mlb = MultiLabelBinarizer()
mlb.fit([termToken])
print(indices)
print(mlb.transform([indices]))

For testing purposes, we only want to consider the 100 most common GOs

In [ ]:
topGOs= item_counts[0:10]
topGOs=topGOs.index.to_list()

mlb = MultiLabelBinarizer()
mlb.fit([topGOs])
print(indices)
print(mlb.transform([indices]))
print(len(mlb.classes_))

## Amino acids encoding

In [ ]:
aminos_list = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'X']

- A: Alanine
- C: Cysteine
- D: Aspartic acid
- E: Glutamic acid
- F: Phenylalanine
- G: Glycine
- H: Histidine
- I: Isoleucine
- K: Lysine
- L: Leucine
- M: Methionine
- N: Asparagine
- O: Pyrrolysine
- P: Proline
- Q: Glutamine
- R: Arginine
- S: Serine
- T: Threonine
- U: Selenocystein
- V: Valine
- W: Tryptophan
- Y: Tyrosine
- X: unknown

In [ ]:
aa_dict = {'A': 1, 'B':24, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'K': 9, 'L': 10, 'M': 11, 'N': 12, 'O': 21, 'P': 13, 'Q': 14, 'R': 15, 'S': 16, 'T': 17, 'U': 22, 'V': 18, 'W': 19, 'Y': 20, 'X':30, 'Z':23}

## Build Dataset

In [ ]:
seqLengths = [len(seq) for seq in sequences]
maxLen = max(seqLengths)
print("The max. length of the sequences is {}".format(maxLen))

In [ ]:
import warnings

TRAIN_VAL_SPLIT = 0.7

#Use numpy vectorize to speed up the mapping (hopefully)
mapping = lambda x: aa_dict[x]
vectMapping = np.vectorize(mapping)

# Shuffle the data
import random
c = list(zip(sequences, ids))
random.shuffle(c)
sequencesShuffle, idsShuffle = zip(*c)

#reduce data for now
sequencesShuffle = sequencesShuffle[0:2000]
idsShuffle = idsShuffle[0:2000]

#Train Validation Split
split = int(np.floor(len(sequencesShuffle)*TRAIN_VAL_SPLIT))
print(split)
trainSeq = sequencesShuffle[0:split]
valSeq = sequencesShuffle[split+1:]
trainIds = idsShuffle[0:split]
valIds = idsShuffle[split+1:]


def generator(padding=True):
    for i,seq in enumerate(trainSeq):
        entryId = trainIds[i]
        labelData = df.loc[df['EntryID'] == entryId]
        
        # indices = labelData["termToken"].to_numpy()
        indices = labelData["term"].to_numpy()

        with warnings.catch_warnings():
            #supress the warnings for unknown classes
            warnings.simplefilter("ignore")
            y = mlb.transform([indices])
        
        arr = np.array(seq)
        mappedArr = vectMapping(arr)
        if padding:
            padWidth = maxLen - arr.size
            mappedArr = np.pad(mappedArr, (0, padWidth))
        yield mappedArr,y[0]

def generatorVal(padding=True):
    for i,seq in enumerate(valSeq):
        entryId = valIds[i]
        labelData = df.loc[df['EntryID'] == entryId]
        
        # indices = labelData["termToken"].to_numpy()
        indices = labelData["term"].to_numpy()

        with warnings.catch_warnings():
            #supress the warnings for unknown classes
            warnings.simplefilter("ignore")
            y = mlb.transform([indices])
        
        arr = np.array(seq)
        mappedArr = vectMapping(arr)
        if padding:
            padWidth = maxLen - arr.size
            mappedArr = np.pad(mappedArr, (0, padWidth))
        yield mappedArr,y[0]
        

In [ ]:
g = generator()
test = next(g)
print("The first sample sequence: {}".format(test[0]))
print("The first sample has {} classes".format(np.count_nonzero(test[1])))

## Basic classification

In [ ]:
X=[]
y=[]
for i,el in enumerate(g):
    X.append(el[0])
    y.append(el[1])
    if i ==10:
        break

In [ ]:

X= np.array(X)
y= np.array(y)
print(X.shape)
print(y.shape)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
# clf.fit(X, y)

# print(clf.score(X,y))
# print(clf.decision_path([X[0]]))
# print(clf.predict([X[0]]))

## Tensorflow Classification

In [ ]:
import tensorflow as tf


dataset = tf.data.Dataset.from_generator(generator, output_signature=(
         tf.TensorSpec(shape=(maxLen,), dtype=tf.int32),
         tf.TensorSpec(shape=(len(mlb.classes_),), dtype=tf.int32)))
print(list(dataset.take(1)))

datasetVal = tf.data.Dataset.from_generator(generatorVal, output_signature=(
         tf.TensorSpec(shape=(None,), dtype=tf.int32),
         tf.TensorSpec(shape=(len(mlb.classes_),), dtype=tf.int32)))

In [ ]:
%load_ext tensorboard


In [ ]:
from tensorflow.keras import layers

VOCAB_SIZE=len(aa_dict)
EMBED_DIM=100

def createModel():
    inputs = tf.keras.Input(shape=(maxLen,1))
    # x=layers.Embedding(VOCAB_SIZE, EMBED_DIM, name="embedding")(inputs)
    # x=layers.GlobalAveragePooling1D()(x)
    # x=layers.Reshape((EMBED_DIM,1))(x)
    x=layers.Conv1D(8, 7)(inputs)
    x=layers.Conv1D(8, 7)(x)
    x=layers.Conv1D(8, 7)(x)
    x=layers.Conv1D(16, 7)(x)
    x=layers.Conv1D(16, 7)(x)
    x=layers.Conv1D(16, 7, strides=2)(x)
    x=layers.Conv1D(16, 7, strides=2)(x)
    x=layers.Conv1D(16, 7, strides=2)(x)
    x=layers.Conv1D(16, 7, strides=2)(x)
    x=layers.Conv1D(16, 7, strides=2)(x)
    x=layers.Conv1D(16, 7, strides=2)(x)
    # x=layers.Conv1D(32, 5, activation=tf.keras.activations.relu)(x)
    # x=layers.Conv1D(32, 5, activation=tf.keras.activations.relu)(x)
    # x=layers.Conv1D(32, 5, activation=tf.keras.activations.relu)(x)
    x=layers.Flatten()(x)
    x=layers.Dense(16)(x)
    x=layers.LeakyReLU()(x)
    outputs=layers.Dense(len(mlb.classes_), activation=tf.keras.activations.sigmoid)(x)
    # outputs=layers.Softmax()(x)

    return tf.keras.Model(inputs=inputs, outputs=outputs, name="embedConvModel")

# model = createModel()

# model.summary()


In [ ]:

VOCAB_SIZE=len(aa_dict)
EMBED_DIM=100

def createRnnModel():
    inputs = tf.keras.Input(shape=(maxLen,))
    x = tf.keras.layers.Masking(0)(inputs)
    x=layers.Embedding(VOCAB_SIZE, EMBED_DIM, name="embedding")(inputs)

    # x = layers.Bidirectional(layers.LSTM(32, return_sequences=True))(x)
    x = layers.Bidirectional(layers.GRU(16, return_sequences=True))(x)
    x = layers.Bidirectional(layers.GRU(16))(x)
    # x = layers.LSTM(32)(x)
    x = layers.Dense(16)(x)
    x = layers.LeakyReLU()(x)
    outputs=layers.Dense(len(mlb.classes_), activation=tf.keras.activations.sigmoid)(x)
    # outputs=layers.Softmax()(x)

    return tf.keras.Model(inputs=inputs, outputs=outputs, name="embedRnnModel")

model = createRnnModel()

model.summary()

In [ ]:
import matplotlib.pyplot as plt
#Learning rate schedule
initial_learning_rate = 0.01
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=10,
    decay_rate=0.9,
    staircase=False)
step = np.linspace(0,1000)
lr = lr_schedule(step)
plt.figure(figsize = (8,6))
plt.yscale("log")
plt.plot(step, lr)
plt.ylim([0,max(plt.ylim())])
plt.xlabel('step')
_ = plt.ylabel('Learning Rate')

In [ ]:
BATCH_SIZE=16
LOG_INTERVAL=5
epochs = 2



print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

log_dir = "./logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1,
                                                      write_graph=True, update_freq=5)


summary_writer = tf.summary.create_file_writer(log_dir)

# Instantiate an optimizer .
# optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule)
# optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# Instantiate a loss function.
loss_fn = tf.keras.losses.CategoricalCrossentropy()

train_acc_metric = tf.keras.metrics.CategoricalAccuracy()
val_acc_metric = tf.keras.metrics.CategoricalAccuracy()

batchedDataset = dataset.batch(BATCH_SIZE, drop_remainder=False)
batchedDatasetVal = datasetVal.batch(BATCH_SIZE, drop_remainder=False)
# print(batchedDataset.take(1))

# @tf.function(jit_compile=True)
def trainStep(x_batch_train, y_batch_train):
    # Open a GradientTape to record the operations run
    # during the forward pass, which enables auto-differentiation.
    with tf.GradientTape() as tape:

        # Run the forward pass of the layer.
        # The operations that the layer applies
        # to its inputs are going to be recorded
        # on the GradientTape.
        probs = model(x_batch_train, training=True) 

        # Compute the loss value for this minibatch.
        loss_value = loss_fn(y_batch_train, probs)

    # Use the gradient tape to automatically retrieve
    # the gradients of the trainable variables with respect to the loss.
    grads = tape.gradient(loss_value, model.trainable_weights)
    #Gradient clipping
    grads = [tf.clip_by_norm(g, 2.0) for g in grads]

    # Run one step of gradient descent by updating
    # the value of the variables to minimize the loss.
    optimizer.apply_gradients(zip(grads, model.trainable_weights))

    train_acc_metric.update_state(y_batch_train, probs)
    return loss_value, grads

maxStep=0

for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch+1,))

    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(batchedDataset):

        loss_value, grads=trainStep(x_batch_train,y_batch_train)

        # Log 
        if step % LOG_INTERVAL == 0:
            template = 'Epoch {}/Step {}, Loss: {:.4f}, Accuracy: {:.4f}, lr schedule: {:.5f}'
            print (template.format(epoch+1, step,loss_value, 
                                    train_acc_metric.result(), lr_schedule(step)))
            print([tf.norm(grad, ord=2).numpy() for grad in grads])
            with summary_writer.as_default():
                tf.summary.scalar('loss', loss_value, step=maxStep*epoch+step)
                tf.summary.scalar('accuracy', train_acc_metric.result(), step=maxStep*epoch+step)
                summary_writer.flush()

    
    train_acc_metric.reset_states()

    maxStep=step

    print("Epoch finished. Start validation")
    for x_batch_val, y_batch_val in batchedDatasetVal:
        valProbs = model(x_batch_val, training=False)
        # Update val metrics
        # val_acc_metric.update_state(y_batch_val, valProbs)
    val_acc = val_acc_metric.result()
    val_acc_metric.reset_states()
    print("Validation acc: %.4f" % (float(val_acc),))
    with summary_writer.as_default():
        tf.summary.scalar('valAcc', float(val_acc), step=epoch)
        summary_writer.flush()